In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Set the style for our visualizations
sns.set_style("whitegrid")
sns.set_palette("deep")

%matplotlib inline

In [ ]:
# Some utility functions for plotting

def plot_eigenvectors(dataset, w, v):
    plt.plot(dataset["x1"], dataset["x2"], "bo", markersize=5)
    plt.arrow(0, 0, 10 * v[0, 0], 10 * v[1, 0], color="r", linewidth=2, head_width=1, head_length=1)
    plt.arrow( 0, 0, 10 * v[0, 1], 10 * v[1, 1], color="r", linewidth=2, head_width=1, head_length=1)
    plt.text( 12 * v[0, 0], 10 * v[1, 0], r"PC1, $\vec{v_1}$ =  %.2f $\vec{x_1}$ + %.2f $\vec{x_2}$" % (v[0, 0], v[1, 0]), fontsize=15)
    plt.text( 26 * v[0, 1], 8 * v[1, 1], r"PC2, $\vec{v_2}$ =  %.2f $\vec{x_&}$ + %.2f $\vec{x_2}$" % (v[0, 1], v[1, 1]), fontsize=15)


def plot_scatter_annotated(dataset, labels):
    X = dataset.values
    plt.figure(figsize=(12, 12))
    plt.scatter(X[:, 0], X[:, 1], s=100)
    for i, (x, y) in enumerate(zip(X[:, 0], X[:, 1])):
        plt.annotate(
            labels[i],
            xy=(x, y),
            xytext=(-20, 20),
            textcoords="offset points",
            ha="right",
            va="bottom",
            bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.5),
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0"),
        )

# 2. Dimensionality reduction


## 2.1 Theory behind PCA

Dimensionality reduction is often useful for visualizing data and improving model performance by reducing overfitting and computational requirements. By removing noisy or correlated features, we can simplify the data while preserving as much relevant information as possible. When reducing the number of features to fewer than four, the data can often be visualized effectively, for example, in a scatterplot.

In this context, we will first focus on **Principal Component Analysis (PCA)**, a popular **feature extraction** technique. PCA transforms the original data into a new set of features, known as principal components (PCs), which are orthogonal to each other and point in the direction of maximum variance.


PCA decomposes a data set in its principal components (PC) through a **linear** transformation. PC's (also called eigenvectors) are **orthogonal** to each other and point in the direction of **largest variance**. So what does this mean? Let's say we have a data set with two features and we want to reduce this data to one feature.

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/notebooks/8_dimensionality_reduction/pca.csv')
sns.lmplot(x="x1", y="x2", data=dataset, fit_reg=False, height=8, scatter_kws={"s": 80})
plt.show()

To apply PCA the data needs to be **centered** first. In this case it was centered already. PCA will search for the direction that preserves the most of the original variance. This is vector PC1 in the plot below.

In [ ]:
w,v=np.linalg.eig(np.cov(dataset.values.T)) #finds the eigenvalues and principle components (eigenvectors)

plt.figure(figsize=(8,8))
plot_eigenvectors(dataset,w,v)
plt.show()

The second principal component (PC2) is orthogonal to the first (PC1). Together, PC1 and PC2 define a new space where PC1 captures most of the variance in the data. In general, for an $n$-dimensional dataset, PCA computes $n$ principal components, where each component captures the maximum variance while being orthogonal to the previous ones.

Each $i$-th principal component has an **eigenvalue** $w_i$, which represents the amount of variance explained by that component.


In [ ]:
print("Eigenvalue for PC1 (w_0): {:.2f}".format(w[0]))
print("Eigenvalue for PC2 (w_1): {:.2f}".format(w[1]))

The amount of variance explained by each PC $i$ is given by

$$\frac{w_i}{\sum_j w_j},$$

where $j$ ranges over all PCs.

In [ ]:
var_explained=100*w/sum(w)
print("Variance explained by PC1: {}".format(var_explained[0]))
print("Variance explained by PC2: {}".format(var_explained[1]))

The first principal component (PC1) explains most of the variance in the original dataset. To reduce the number of dimensions, we can remove PC2 and project all data points onto PC1, which preserves 91.78% of the variance. In contrast, projecting onto PC2 alone preserves only 8.22%. Thus, to visualize an $n$-dimensional dataset, we can project it onto the top two or three principal components.

For example, if we remove PC2 and project the data onto PC1, we can compute the new feature values from the original features $x_1$ and $x_2$ as:

$$\vec{v_1} =  0.85 \vec{x_1} + 0.53 \vec{x_2}.$$

Here, we replace the original features $x_1$ and $x_2$ with the new feature $v_1$, which can then be plotted as a histogram.


In [ ]:
plt.figure(figsize=(7, 7))
sns.histplot(v[0, 0] * dataset["x1"] + v[1, 0] * dataset["x2"])
plt.show()

The variance of this distribution is thus 38.81. To show how PCA is applied using the PCA module in scikit-learn we first create an artificial data set with 100 samples in two classes. There are 100 features of which 10 are informative:

In [ ]:
import sklearn.datasets as datasets

(dataset_big,targets) = datasets.make_classification(
n_samples=100,
n_features=100,
n_classes=2,
n_informative=10,
n_redundant=0,
n_repeated=0,
class_sep=2,
n_clusters_per_class=1,
random_state=2,
)

Remember that this dataset is 100-dimensional, which makes it impossible to visualize directly.

Now we apply PCA and keep only the two top principle components. The data set is projected (or transformed) onto these two new dimensions and plotted:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
dataset_big_projected = pd.DataFrame(pca.fit(dataset_big).transform(dataset_big),
                                     columns=['PC1','PC2'])
dataset_big_projected['label'] = targets

sns.lmplot(x="PC1", y="PC2", data=dataset_big_projected, hue='label', markers=['+','o'],
           fit_reg=False, height=7, scatter_kws={"s": 80})
plt.show()

The plots reveals the two classes (remember these were not known to the PCA algorithm) and also reveals possible outliers in both classes. We can look at the explained variance for each PC:

In [ ]:
print("Variance explained by PC1: {}".format(pca.explained_variance_ratio_[0]))
print("Variance explained by PC2: {}".format(pca.explained_variance_ratio_[1]))

We observe how PC1 and PC2 together only explain about 27% of the original variance while still providing useful structural information about the 100-dimensional data set.

## 2.2 Applying PCA, t-SNA, and UMAP to a biological dataset

In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import umap
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE

## Dataset Description

**Study:** *Feminizing gender-affirming hormone therapy remodels the plasma proteome*

**Source:** 
- Article: [Nature Medicine (2025)](https://www.nature.com/articles/s41591-025-04023-9)
- Data: [PRIDE Archive Project PAD000016](https://www.ebi.ac.uk/pride/archive/projects/PAD000016)

**Abstract:**

Sex differences manifest in various traits, as well as in the risk of cardiovascular, metabolic and immunological conditions. Despite the clear physical changes induced by gender-affirming hormone therapy (GAHT), little is known about how it affects underlying physiological and biochemical processes. Here we examined plasma proteome changes over 6 months of feminizing GAHT in 40 transgender individuals treated with estradiol plus one of two antiandrogens: cyproterone acetate or spironolactone. Testosterone levels dropped markedly in the cyproterone group, but less so in those receiving spironolactone. Among 5,279 total proteins measured, feminizing GAHT changed the levels of 245 and 91, in the cyproterone and spironolactone groups, respectively, with most (>95%) showing a decrease. Proteins associated with male spermatogenesis showed a marked decrease in the cyproterone group, attributable specifically to loss of testosterone. Changes in body fat percentage and breast volume following GAHT were also reflected in the plasma proteome, including an increase in leptin expression. We show that feminizing GAHT remodels the proteome toward a cis-female profile, altering 36 (cyproterone) and 22 (spironolactone) of the top 100 sex-associated proteins in UK Biobank adult data. Moreover, 43% of cyproterone-affected proteins overlapped with those altered by menopausal hormone therapy in cis women, showing the same directional changes, with notable exceptions including CXCL13 and NOS3. Feminizing GAHT skewed the protein profile toward that linked to asthma and autoimmunity, while GAHT with cyproterone specifically skewed it away from an atherosclerosis-associated profile, suggesting a protective effect. These results reveal that feminizing GAHT reshapes the plasma proteome in a hormone-dependent manner, with implications for reproductive capacity, immune regulation and long-term health outcomes.

**Dataset Details:**

- **Samples:** 113 plasma samples from transgender individuals undergoing feminizing GAHT, cis-female controls, cis-male controls, and pregnant individuals
- **Features:** 5,440 proteins measured using proximity extension assay (PEA) technology
- **Phenotype Groups:**
  - SPIRO_GAHT (spironolactone + estradiol): baseline and 6-month timepoints
  - CPA_GAHT (cyproterone acetate + estradiol): baseline and 6-month timepoints
  - Control groups: cis-female and cis-male at baseline and follow-up timepoints
  - Pregnancy: trimester 1 and trimester 3 samples
- **Data Format:** Protein expression values are reported as PCNormalizedNPX values

**Analysis Goal:**

In this notebook, we apply three dimensionality reduction techniques—**PCA** (Principal Component Analysis), **t-SNE** (t-distributed Stochastic Neighbor Embedding), and **UMAP** (Uniform Manifold Approximation and Projection)—to visualize how the plasma proteome differs across phenotype groups and to identify key proteins driving these differences.

In [ ]:
# Load the datasets
data_file = "PAD000016_data.csv"
metadata_file = "GAHT_pregnancy_metadata_SDRF.tsv"

# Read the CSV and TSV files
data_df = # FILL IN
metadata_df = # FILL IN

# Extracting the phenotype labels from the metadata
phenotype_labels = # FILL IN

# Pivoting the data into wide format if needed (as in previous steps)
data_wide = # FILL IN

# Filling missing values with a set value
data_wide_filled_missing_values = # FILL IN

# Align phenotype labels with the sample_subset index
# Match the metadata sample IDs with the data sample IDs
phenotype_labels_aligned = metadata_df.set_index("source name")["factor value[phenotype]"].reindex(
    data_wide_filled_missing_values.index, fill_value="Unknown"
)

Question 1 - Do we need the labels for dimensionality reduction?

Question 2 - Here we do not normalize the data, is normalization neccesary? If yes, why? If yes, what kind of normalization?

Next we will fit the models to perform the dimensionality reduction. You can always look at the documentation to see how the models should be fitted [scikit-learn](https://scikit-learn.org/stable/) and [UMAP](https://umap-learn.readthedocs.io/en/latest/).

In [ ]:
# Perform PCA
pca = # FILL IN
pca_result = # FILL IN

# Perform t-SNE with parallelization and adjusted perplexity for performance
tsne = # FILL IN
tsne_result = # FILL IN

# Perform UMAP (ensure umap-learn is installed)
umap_model = # FILL IN
umap_result = # FILL IN

Next we plot the results from the different dimensionality reduction models.

In [ ]:
# Plotting PCA, t-SNE, and UMAP results
fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# PCA plot
sns.scatterplot(
    x=pca_result[:, 0],
    y=pca_result[:, 1],
    hue=phenotype_labels_aligned,
    ax=axes[0],
    palette="viridis",
    legend=False,
)
axes[0].set_title("PCA")

# t-SNE plot
sns.scatterplot(
    x=tsne_result[:, 0],
    y=tsne_result[:, 1],
    hue=phenotype_labels_aligned,
    ax=axes[1],
    palette="viridis",
    legend=False,
)
axes[1].set_title("t-SNE")

# UMAP plot
sns.scatterplot(
    x=umap_result[:, 0],
    y=umap_result[:, 1],
    hue=phenotype_labels_aligned,
    ax=axes[2],
    palette="viridis",
)
axes[2].set_title("UMAP")

# Move legend outside the plot area to the right
axes[2].legend(
    bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0, frameon=True
)

plt.tight_layout()
plt.show()

Question 3 - What are the differences between the different algorithms? Are the differences you see expected for PCA/t-SNE/UMAP?

Question 4 - What could this visualization be useful for?

In [ ]:
# Test different perplexity values
perplexity_values = # FILL IN

fig, axes = plt.subplots(math.ceil(len(perplexity_values) / 3), 3, figsize=(20, 12))
axes = axes.flatten()

for idx, perp in enumerate(perplexity_values):
    # Run t-SNE with different perplexity
    tsne_temp = # FILL IN
    tsne_temp_result = # FILL IN

    # Scatter plot of t-SNE
    sns.scatterplot(
        x=tsne_temp_result[:, 0],
        y=tsne_temp_result[:, 1],
        hue=phenotype_labels_aligned,
        ax=axes[idx],
        palette="viridis",
        legend=(idx == 0),
        s=60,
        alpha=0.7,
    )

    axes[idx].set_title(
        f"t-SNE with Perplexity = {perp}", fontsize=14, fontweight="bold"
    )
    axes[idx].set_xlabel("t-SNE Dimension 1", fontsize=11)
    axes[idx].set_ylabel("t-SNE Dimension 2", fontsize=11)

    if idx == 0:
        axes[idx].legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)

plt.tight_layout()
plt.show()

Question 5 - How do different perplexity values have an effect on the dimensionality reduction?

Question 6 - How do you select an appropiate perplexity value?

## Feature Importances and Interpretation

For PCA, we can examine the **loadings** (also called components or feature weights) which tell us how much each original feature (protein) contributes to each principal component. Features with large absolute loadings have the most influence on that PC.

In [ ]:
# PCA Feature Importances (Loadings)
# Get the loadings for PC1 and PC2
loadings = pd.DataFrame(
    # FILL IN
    columns=["PC1", "PC2"],
    index=# FILL IN,
)

# Calculate the absolute importance (magnitude of loading)
loadings["PC1_abs"] = np.abs(loadings["PC1"])
loadings["PC2_abs"] = np.abs(loadings["PC2"])

# Get top 15 features for each PC
top_pc1 = loadings.nlargest(# FILL IN, "PC1_abs")[["PC1"]].sort_values("PC1")
top_pc2 = loadings.nlargest(# FILL IN, "PC2_abs")[["PC2"]].sort_values("PC2")

print(
    "Variance explained by PC1: {:.2f}%".format(pca.explained_variance_ratio_[0] * 100)
)
print(
    "Variance explained by PC2: {:.2f}%".format(pca.explained_variance_ratio_[1] * 100)
)
print(
    "Total variance explained for the top 15: {:.2f}%".format(sum(pca.explained_variance_ratio_) * 100)
)

Question 7 - Is the variance explained by PC1 large when considering the number of features? Is there a steep drop-off in variance explained? Is this expected?

In [ ]:
# Visualize top feature loadings for PC1 and PC2
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PC1 loadings
top_pc1.plot(kind="barh", ax=axes[0], color="steelblue", legend=False)
axes[0].set_title("Top 15 Feature Loadings for PC1", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Loading Value", fontsize=12)
axes[0].set_ylabel("Protein/Feature", fontsize=12)
axes[0].axvline(x=0, color="black", linestyle="-", linewidth=0.5)

# PC2 loadings
# FILL IN

plt.tight_layout()
plt.show()

Question 8 - What are the top 5 most important proteins for PC1 and PC2, respectively?

Question 9 - Can the loadings between different principal components overlap? What does it indicate if a protein has a high loading for the first couple of components? How can we account for this spread of loadings?

In [ ]:
# Get top 5 feature names for PC1
top_5_pc1_features = # FILL IN

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for idx, feature in enumerate(top_5_pc1_features):
    ax = axes[idx]

    # Scatter plot: Feature vs PC1
    scatter = ax.scatter(
        # FILL IN,
        pca_result[:, 0],
        c=pd.Categorical(phenotype_labels_aligned).codes,
        cmap="viridis",
        s=60,
        alpha=0.7,
        edgecolors="black",
        linewidth=0.5,
    )

    # Calculate correlation
    corr = # FILL IN https://numpy.org/doc/2.3/reference/generated/numpy.corrcoef.html

    # Add regression line using sklearn
    X = data_wide_filled_missing_values[feature].values.reshape(-1, 1)
    y = pca_result[:, 0]
    reg = LinearRegression().fit(X, y)

    line_x = np.array(
        [
            data_wide_filled_missing_values[feature].min(),
            data_wide_filled_missing_values[feature].max(),
        ]
    ).reshape(-1, 1)

    line_y = reg.predict(line_x)

    ax.plot(line_x, line_y, "r--", linewidth=2, alpha=0.7)

    # Formatting
    feature_name = feature.replace("_", " ")
    ax.set_xlabel(f"{feature_name} Expression", fontsize=10)
    ax.set_ylabel("PC1 Score", fontsize=10)
    ax.set_title(
        f'{feature_name}\nLoading={loadings.loc[feature, "PC1"]:.4f}, r={corr:.3f}',
        fontsize=11,
        fontweight="bold",
    )
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Question 10 - Does a high loading correlate with a good split between the classes? Why not? why yes?

Question 11 - Investigate some of the proteins and their implications on pregnancy and hormone therapy, are there any relations?

Questions 12 - What would be a next step in data analysis after reducing the dimensions?